<a href="https://colab.research.google.com/github/kavyakapoor200/Sales-Caller-AI/blob/main/Sales_Caller_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask twilio transformers accelerate sentencepiece pyngrok


In [2]:
from huggingface_hub import login
login(token="your hugging face api")  # Replace with your actual Hugging Face token


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Correct model name
model_name = "mistralai/Mistral-7B-Instruct-v0.1"
hf_token = "your hugging face api"  # Replace with your token

print(" Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token)

print(" Loading model (This may take time)...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    token=hf_token  # Ensure the token is passed
)

print(" AI Model Loaded Successfully!")


🔄 Loading tokenizer...
🔄 Loading model (This may take time)...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ AI Model Loaded Successfully!


In [4]:
import threading
import time
import requests
from twilio.rest import Client
from flask import Flask, request
from twilio.twiml.voice_response import VoiceResponse, Gather
from pyngrok import ngrok

# Twilio Credentials
TWILIO_SID = "xxxxxxxxxxxxxxxxxxxxx"
TWILIO_AUTH_TOKEN = "zzzzzzzzzzzzzzzzzzz"
TWILIO_PHONE = "bbbbbbbbbbbbbbb"
CUSTOMER_PHONE = "cccccccccccccccc"

# Initialize Twilio Client
client = Client(TWILIO_SID, TWILIO_AUTH_TOKEN)

# Flask App
app = Flask(__name__)

@app.route("/voice", methods=["POST"])
def voice():
    response = VoiceResponse()
    response.say("Hello! Welcome to our AI-powered sales assistant.", voice="alice")

    gather = Gather(num_digits=1, action="/process_response", timeout=5)
    gather.say("Press 1 for course details. Press 2 to book a Google Meet demo.")
    response.append(gather)

    return str(response)

@app.route("/process_response", methods=["POST"])
def process_response():
    response = VoiceResponse()
    digits = request.form.get("Digits")

    if digits == "1":
        response.say("Our AI course covers Machine Learning, Deep Learning, and NLP. The cost is 10,000 INR.", voice="alice")
    elif digits == "2":
        response.say("Your Google Meet demo has been scheduled. You will receive an SMS with the link.", voice="alice")
    else:
        response.say("Invalid response. Please try again.")

    return str(response)

# Function to Start Flask
def start_flask():
    print(" Starting Flask Server...")
    app.run(debug=True, port=5001, use_reloader=False)

# Function to Start ngrok and Get URL
def start_ngrok():
    print("Starting ngrok...")
    public_url = ngrok.connect(5001).public_url
    print(f" ngrok is running at: {public_url}")
    return public_url

# Function to Update Twilio with ngrok URL & Make Call
def make_twilio_call():
    time.sleep(5)  # Allow Flask to start first
    ngrok_url = start_ngrok()  # Automatically get ngrok URL

    print(" Updating Twilio with the latest ngrok URL...")
    call = client.calls.create(
        to=CUSTOMER_PHONE,
        from_=TWILIO_PHONE,
        url=f"{ngrok_url}/voice"  # No need to update manually
    )
    print(f" Call initiated: {call.sid}")

# Run Flask, ngrok, and Twilio Call in Parallel
threading.Thread(target=start_flask).start()
threading.Thread(target=make_twilio_call).start()


🚀 Starting Flask Server...
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5001
INFO:werkzeug:Press CTRL+C to quit
